In [1]:
import molsysmt as msm

In [2]:
molecular_system = msm.convert(msm.systems.demo['Barnase-Barstar']['barnase_barstar.msmpk'])

In [3]:
modeller = msm.convert(molecular_system, to_form='openmm.Modeller')

In [4]:
import openmm as mm
from openmm import unit
from openmm import app

In [5]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [6]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [7]:
#forcegroups = {}
#for ii in range(system.getNumForces()):
#    force = system.getForce(ii)
#    print(f'{force.getName()} was in force group {force.getForceGroup()} and it is now in group {ii}')
#    force.setForceGroup(ii)
#    forcegroups[force] = ii

In [8]:
# A simulation object is needed

## Integrator
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

## Platform
platform_name = 'CUDA'
platform    = mm.Platform.getPlatformByName(platform_name)

In [9]:
# Creación del objeto context del sistema en vacio
context = mm.Context(system, integrator, platform)

In [10]:
# Condiciones iniciales
context.setPositions(modeller.positions)

In [11]:
# Minimizacion del sistema
state_pre_minimization = context.getState(getEnergy=True)
mm.LocalEnergyMinimizer.minimize(context)
state_post_minimization = context.getState(getEnergy=True)

In [12]:
print('Energy before minimization:', state_pre_minimization.getPotentialEnergy())
print('Energy after minimization:', state_post_minimization.getPotentialEnergy())

Energy before minimization: 18582.798828125 kJ/mol
Energy after minimization: -24856.564453125 kJ/mol


In [13]:
tmp_system = context.getSystem()
forcegroups = {}
for ii in range(tmp_system.getNumForces()):
    force = system.getForce(ii)
    print(f'{force.getName()} was in force group {force.getForceGroup()} and it is now in group {ii}')
    force.setForceGroup(ii)
    forcegroups[force] = ii


tmp_context = mm.Context(tmp_system, mm.VerletIntegrator(0.001), context.getPlatform())
tmp_context.setPositions(context.getState(getPositions=True).getPositions())
energies = {}
for ff, ii in forcegroups.items():
    energies[ff.getName()] = tmp_context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

HarmonicBondForce was in force group 0 and it is now in group 0
NonbondedForce was in force group 0 and it is now in group 1
PeriodicTorsionForce was in force group 0 and it is now in group 2
CMMotionRemover was in force group 0 and it is now in group 3
HarmonicAngleForce was in force group 0 and it is now in group 4


In [14]:
for ii,jj in energies.items():
    print(ii,jj)

HarmonicBondForce -24856.564453125 kJ/mol
NonbondedForce 0.0 kJ/mol
PeriodicTorsionForce 0.0 kJ/mol
CMMotionRemover 0.0 kJ/mol
HarmonicAngleForce 0.0 kJ/mol


In [ ]:
context.getState(getPositions=True).getPositions()